In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import cross_val_predict

# part1: Data Preprocessing

In [ ]:
df=pd.read_csv("student4.csv")

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
def categorize_age(age):
    if age <= 18:
        return 'Teenager'
    elif age <= 22:
        return 'Young'
  
    else:
        return -1


df['Category_Age'] = df['age'].apply(categorize_age)

In [ ]:
df = pd.get_dummies(df,columns=['Category_Age','school','sex','address','famsize','Pstatus','schoolsup','famsup','paid','activities','nursery','higher','internet','romantic','Mjob','Fjob','reason','guardian'],dtype=int)


In [ ]:
df.drop('age', axis=1, inplace=True)

In [ ]:
numeric_features = df.columns[df.dtypes != 'object']
binary_features = ['binary_feature_1', 'binary_feature_2', ..., 'binary_feature_17']
numeric_features = [feature for feature in numeric_features if feature not in binary_features]
numeric_df = df[numeric_features]
scaler = MinMaxScaler()
df[numeric_features] = scaler.fit_transform(numeric_df)

In [ ]:
df.describe() 

# PART 1: Data Visualization 


In [ ]:
plt.figure(figsize=(60, 70))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', linewidths=.5)

In [ ]:
plt.boxplot(df['G3'])
plt.title('Box Plot for Feature')
plt.xlabel('Feature')
plt.ylabel('Values')
plt.show()

In [ ]:
df.plot(kind="scatter", x="G3", y="G2")

In [ ]:
plt.boxplot(df['G2'])
plt.title('Box Plot for Feature')
plt.xlabel('Feature')
plt.ylabel('Values')
plt.show()

In [ ]:
plt.boxplot(df['G1'])
plt.title('Box Plot for Feature')
plt.xlabel('Feature')
plt.ylabel('Values')
plt.show()

In [ ]:
selected_features = ['Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'Dalc', 'absences', 'G1', 'G2', 'Category_Age_Teenager']
plt.figure(figsize=(20, 10))   
sns.boxplot(data=df[selected_features])
plt.title(f"Boxplot for Selected Features")
plt.show()

In [ ]:
df.plot(kind="scatter", x="G2", y="absences")

In [ ]:
selected_features = ['Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'Dalc', 'absences', 'G1', 'G2', 'Category_Age_Teenager']
pd.plotting.scatter_matrix (df[selected_features], figsize = (26,24), diagonal = '')                                          
plt.show()

In [ ]:
new_corr=df[['G3','G2','G1']].corr()

In [ ]:
plt.figure(figsize=(12, 10))
sns.heatmap(new_corr, annot=True, cmap='coolwarm', linewidths=.5)

# PART 3: Regression hold_out_method

In [ ]:
x = df.drop(['G3'], axis=1)
y = df['G3']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
lr = LinearRegression()
lr.fit(x_train, y_train)
pred = lr.predict(x_test)
pred2 = lr.predict(x_train)
selected_features = []
model_performance = {}
for feature in x_train.columns:
    selected_features.append(feature)
    model = LinearRegression()
    model.fit(x_train[selected_features], y_train)    
    y_pred = model.predict(x_test[selected_features])    
    r2 = r2_score(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)   
    model_performance[", ".join(selected_features)] = (r2, mse)    
    if len(selected_features) > 1 and r2 < max(model_performance.values())[0]:
        selected_features.pop()

In [ ]:
best_features = ""
best_r2 = 0
best_mse = 1000                
for features, performance in model_performance.items():
    print("Selected Features:", features)
    print("R-squared Score:", performance[0])
    print("Mean Squared Error:", performance[1])
    print('-------------------------------------------------------------------------------------------')
    if performance[0] > best_r2:
        best_features = features
        best_r2 = performance[0]
        best_mse = performance[1]
print("Best Model is:")
print("Selected Features:", best_features)
print("R-squared Score:", best_r2)
print("Mean Squared Error:", best_mse)

In [ ]:
X=df[['Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel', 'Dalc', 'absences', 'G1', 'G2', 'Category_Age_Teenager', 'Category_Age_Young', 'school_GP', 'school_MS', 'sex_F', 'famsize_GT3', 'famsize_LE3', 'famsup_no', 'famsup_yes', 'paid_no', 'internet_no', 'internet_yes', 'romantic_no', 'romantic_yes', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'reason_course', 'guardian_mother', 'guardian_other']]
Y = df['G3']


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
regr = LinearRegression()
regr.fit(X_train, Y_train)
predicted_labels= regr.predict(X_test)
Errors = mean_squared_error(Y_test, regr.predict(X_test))
error=np.sqrt(Errors)
Y_pred =regr.predict(X_test)
r2 = r2_score(Y_test, Y_pred)
print("R2 score:", r2)
print("mean_squared_error=",error)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
rf = RandomForestRegressor()
rf.fit(X_train, Y_train)
pred = rf.predict(X_test)
mse = mean_squared_error(Y_test, pred)
print(mse)
r2 = r2_score(Y_test, pred)
print(r2)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
dt = DecisionTreeRegressor()
dt.fit(X_train, Y_train)
pred = dt.predict(X_test)
mse = mean_squared_error(Y_test, pred)
print(mse)
r2 = r2_score(Y_test, pred)
print(r2)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
ada = AdaBoostRegressor()
ada.fit(X_train, Y_train)
pred = ada.predict(X_test)
mse = mean_squared_error(Y_test, pred)
print(mse)
r2 = r2_score(Y_test, pred)
print(r2)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
svr = SVR()
svr.fit(X_train, Y_train)
pred = svr.predict(X_test)
mse = mean_squared_error(Y_test, pred)
print(mse)
r2 = r2_score(Y_test, pred)
print(r2)

# Part 3 (B):LinearRegression() 10-fold cross validation


In [ ]:
from sklearn.model_selection import KFold

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
num_folds = 10
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
model = LinearRegression()
mse_scores = []
r2_scores = []
for fold, (train_index, test_index) in enumerate(kf.split(X, Y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    # Fit the model
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate MSE and R2
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse_scores.append(mse)
    r2_scores.append(r2)
    print(f"Fold {fold + 1} - MSE: {mse}, R2: {r2}")
# Calculate average MSE and R2 across all folds
avg_mse = sum(mse_scores) / len(mse_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)
print(f"\nAverage MSE: {avg_mse}")
print(f"Average R2: {avg_r2}")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
num_folds = 10
# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
# Initialize RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100, random_state=42)
# Lists to store results
mse_scores = []
r2_scores = []
# Iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(X, Y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    # Fit the model
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate MSE and R2
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse_scores.append(mse)
    r2_scores.append(r2)
    print(f"Fold {fold + 1} - MSE: {mse}, R2: {r2}")
# Calculate average MSE and R2 across all folds
avg_mse = sum(mse_scores) / len(mse_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)
print(f"\nAverage MSE: {avg_mse}")
print(f"Average R2: {avg_r2}")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
num_folds = 10
# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
# Initialize DecisionTreeRegressor model
model = DecisionTreeRegressor(random_state=42)
# Lists to store results
mse_scores = []
r2_scores = []
# Iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(X, Y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    # Fit the model
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate MSE and R2
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse_scores.append(mse)
    r2_scores.append(r2)
    print(f"Fold {fold + 1} - MSE: {mse}, R2: {r2}")
# Calculate average MSE and R2 across all folds
avg_mse = sum(mse_scores) / len(mse_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)
print(f"\nAverage MSE: {avg_mse}")
print(f"Average R2: {avg_r2}")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
num_folds = 10
# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
# Initialize AdaBoostRegressor model
model = AdaBoostRegressor(random_state=42)
# Lists to store results
mse_scores = []
r2_scores = []
# Iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(X, Y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Calculate MSE and R2
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    mse_scores.append(mse)
    r2_scores.append(r2)

    print(f"Fold {fold + 1} - MSE: {mse}, R2: {r2}")

# Calculate average MSE and R2 across all folds
avg_mse = sum(mse_scores) / len(mse_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)

print(f"\nAverage MSE: {avg_mse}")
print(f"Average R2: {avg_r2}")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
num_folds = 10
# Initialize KFold
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
# Initialize SVR model
model = SVR()
# Lists to store results
mse_scores = []
r2_scores = []
# Iterate over the folds
for fold, (train_index, test_index) in enumerate(kf.split(X, Y)):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]
    # Fit the model
    model.fit(X_train, y_train)
    # Make predictions
    y_pred = model.predict(X_test)
    # Calculate MSE and R2
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    mse_scores.append(mse)
    r2_scores.append(r2)
    print(f"Fold {fold + 1} - MSE: {mse}, R2: {r2}")
# Calculate average MSE and R2 across all folds
avg_mse = sum(mse_scores) / len(mse_scores)
avg_r2 = sum(r2_scores) / len(r2_scores)
print(f"\nAverage MSE: {avg_mse}")
print(f"Average R2: {avg_r2}")

# PART 2: Classification

In [ ]:
data = pd.read_csv('student4.csv')

In [ ]:
numeric_features = data.columns[data.dtypes != 'object']
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[numeric_features])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_features = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']
for feature in cat_features:
    data[feature] = le.fit_transform(data[feature])    

In [ ]:
data['pass'] = np.where(data['G3'] >= 10, 'Pass', 'Fail')


In [ ]:
selected_features = ['G2', 'G1','higher', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'studytime', 'famsup', 'nursery','activities','paid','internet','famrel','reason','famsize','address']
X = data[selected_features]
y = data['pass']

# hold-out method

In [ ]:
from sklearn import svm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = svm.SVC( probability=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print('Confusion matrix for hold-out method:')
print(cm)
print('Accuracy score for hold-out method: {:.2f}%'.format(acc*100))

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test) 

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 

y_predict = classifier.predict(X_test)
cm=confusion_matrix(y_test, y_predict)
print('confusion_matrix\n\n', cm) 
print('\nTrue Positives(TP) = ' , cm[0,0])
print('\nTrue Negatives(TN) = ' , cm[1,1]) 
print('\nFalse Positives(FP) = ' , cm[0,1]) 
print('\nFalse Negatives(FN) = ' , cm[1,0])
print(classification_report(y_test, y_predict)) 
print("accuracy=", accuracy_score(y_test, y_predict))

In [ ]:


scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

nb_classifier = GaussianNB()

nb_classifier.fit(X_train, y_train)

y_predict = nb_classifier.predict(X_test)

cm = confusion_matrix(y_test, y_predict)
print('Confusion_Matrix\n\n', cm)

print('\nClassification Report:\n', classification_report(y_test, y_predict))

print("Accuracy:", accuracy_score(y_test, y_predict))


In [ ]:
classifier = KNeighborsClassifier(n_neighbors=5)
classifier.fit(X_train, y_train) 

y_predict = classifier.predict(X_test)

print(confusion_matrix(y_test, y_predict))

print(classification_report(y_test, y_predict)) 

In [ ]:

clf = RandomForestClassifier(n_estimators=100, random_state=42)

clf.fit(X_train, y_train)

Y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test,Y_pred))
print("Classification Report:\n", classification_report(y_test, Y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, Y_pred))


# 10-fold cross validation

In [ ]:

kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')
print('Mean accuracy score for 10-fold cross validation: {:.2f}%'.format(scores.mean()*100))
y_pred = cross_val_predict(clf, X, y, cv=kf)
cm = confusion_matrix(y, y_pred)
print('Confusion matrix for cross validation:')
print(cm)

In [ ]:

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

scores = cross_val_score(classifier, X, y, cv=10)

y_pred = cross_val_predict(classifier, X, y, cv=10)

cm = confusion_matrix(y, y_pred)

print('Confusion_Matrix\n\n', cm)
print('\nTrue Positives(TP) = ', cm[0, 0])
print('\nTrue Negatives(TN) = ', cm[1, 1])
print('\nFalse Positives(FP) = ', cm[0, 1])
print('\nFalse Negatives(FN) = ', cm[1, 0])
print('-------------------------------------------------------')
print(classification_report(y, y_pred))
print('-------------------------------------------------------')
print("Score:", scores)
print("Average Score:", scores.mean())



In [ ]:

classifier = KNeighborsClassifier()

scores = cross_val_score(classifier, X, y, cv=10)

y_pred = cross_val_predict(classifier, X, y, cv=10)

cm = confusion_matrix(y, y_pred)

print('Confusion_Matrix\n\n', cm)
print('\nTrue Positives(TP) = ' , cm[0,0])
print('\nTrue Negatives(TN) = ' , cm[1,1]) 
print('\nFalse Positives(FP) = ' , cm[0,1])
print('\nFalse Negatives(FN) = ' , cm[1,0]) 
print('-------------------------------------------------------')
print(classification_report(y, y_pred)) 
print('-------------------------------------------------------')
print("Score:", scores)
print("Average Score:", scores.mean())
print('-------------------------------------------------------')



In [ ]:


classifier = DecisionTreeClassifier()

scores = cross_val_score(classifier, X, y, cv=10)

y_pred = cross_val_predict(classifier, X, y, cv=10)

cm = confusion_matrix(y, y_pred)
print('Confusion_Matrix\n\n', cm)
print('\nTrue Positives(TP) = ' , cm[0,0])
print('\nTrue Negatives(TN) = ' , cm[1,1]) 
print('\nFalse Positives(FP) = ' , cm[0,1])
print('\nFalse Negatives(FN) = ' , cm[1,0]) 
print('-------------------------------------------------------')

print(classification_report(y, y_pred)) 
print('-------------------------------------------------------')

print("Score:", scores)
print("Average Score:", scores.mean())
print('-------------------------------------------------------')


In [ ]:


nb_classifier = GaussianNB()

scores = cross_val_score(nb_classifier, X, y, cv=10)

y_pred = cross_val_predict(nb_classifier, X, y, cv=10)

cm = confusion_matrix(y, y_pred)
print('Confusion_Matrix\n\n', cm)

print('\nClassification Report:\n', classification_report(y, y_pred))

print('-------------------------------------------------------')
print("Score:", scores)
print("Average Score:", scores.mean())
print('-------------------------------------------------------')
